# Can you crack the face-down card game?

#### Finding all solutions to the [face-down-card-game problem](https://www.youtube.com/watch?v=oCMVUROty0g) by exhaustion.

*You should first watch the video, try to solve it by your brain and maths, then use algorithms to double check.*

In [1]:
def flip_cost(a,b):
    """Find number of bits that are needed to be flipped to get b from a in binary"""
    return format(a^b, 'b').count('1') # a XOR b, count number of 1s

def flip_position(a,b, n=4, cnt_start=1):
    """Find which cards to flip to get b from a"""
    return  [i+cnt_start for i, char in enumerate(format(a^b, f'{n}b')) if char == '1']

def print_strategy(order):
    """Print strategy given an order"""
    for i in range(len(order)-1):
        a,b = order[i], order[i+1]
        print(f'Step {i+1}: flip card {flip_position(a,b)} to get {a} -> {b}')

Every state (or node) is represented by an integer. For instance, 10 is 1010 in binary, which represents up-down-up-down setting in a 4-card game. For an n-card game, there are 2**n different possible states.

Our goal is to visit every state at least once. We don't need to go back to the starting node at the end. The relationships among nodes can be visualised as an n-dimensional hypercube. Going from node a to node b requires one flipping iff they share an edge in the hypercube.

In [2]:
from itertools import permutations
from functools import reduce
def get_order_and_cost(n=4):
    """Get the ideal order and the flipping cost of an n-card game"""
    acc_order, acc_cost = (0,1), 1 # for 1 bit, visiting every node is 0 -> 1 with flipping cost 1
    
    # for n bits, start the flipping order with the best order in (n-1) bit
    # e.g. for 2 bits, start with 0 -> 1 -> ?
    # then we found the best 2-bit flipping order is 0 -> 1 -> 3 -> 2
    # then for 3 bits, start with 0 -> 1 -> 3 -> 2 -> ?
    # and so forth
    for n in range(2,n+1):
        best_order, best_cost = None, 9999
        bits = tuple(range(2**(n-1), 2**n)) # nodes < 2**(n-1) has already visited in the previous loop
        perm = permutations(bits, len(bits)) 

        for _order in list(perm):
            order = (acc_order[-1],) + _order # start from the last node of the pre-determined order
            temp_cost = 0
            for i in range(len(order)-1):
                temp_cost += flip_cost(order[i], order[i+1])
            if temp_cost < best_cost:
                best_order, best_cost = order, temp_cost

        acc_order = (*acc_order,) + best_order[1:]
        acc_cost += best_cost
    return acc_order, acc_cost

In [3]:
acc_order, acc_cost = get_order_and_cost(4)
print(acc_cost, acc_order)

15 (0, 1, 3, 2, 6, 4, 5, 7, 15, 11, 9, 8, 10, 14, 12, 13)


In [4]:
print_strategy(acc_order)

Step 1: flip card [4] to get 0 -> 1
Step 2: flip card [3] to get 1 -> 3
Step 3: flip card [4] to get 3 -> 2
Step 4: flip card [2] to get 2 -> 6
Step 5: flip card [3] to get 6 -> 4
Step 6: flip card [4] to get 4 -> 5
Step 7: flip card [3] to get 5 -> 7
Step 8: flip card [1] to get 7 -> 15
Step 9: flip card [2] to get 15 -> 11
Step 10: flip card [3] to get 11 -> 9
Step 11: flip card [4] to get 9 -> 8
Step 12: flip card [3] to get 8 -> 10
Step 13: flip card [2] to get 10 -> 14
Step 14: flip card [3] to get 14 -> 12
Step 15: flip card [4] to get 12 -> 13
